# Semantic Kernel in Python
[Semantic Kernel](https://github.com/microsoft/semantic-kernel) (SK) は Microsoft が OSS として発表した、大規模言語モデル (LLM) をアプリにすばやく簡単に統合できる SDK です。Semantic Kernel は従来のプログラミング言語と最新のLLM AI "プロンプト" を簡単に組み合わせることができ、テンプレート化、チェーン化、埋め込みベースのメモリー、およびプランニング機能を備えています。

本サンプルコードは、Semantic Kernel 公式 [Notebook](https://github.com/microsoft/semantic-kernel/tree/main/python/notebooks) に基づいています。Microsoft Learn によるドキュメントは[こちら](https://learn.microsoft.com/semantic-kernel/overview/)。

## 事前準備

この Python サンプルを実行するには、以下が必要です：

- Azure OpenAI Service にアクセスできる[承認済み](https://aka.ms/oai/access) Azure サブスクリプション
- Azure OpenAI Service への GPT-3.5 Turbo モデルのデプロイメント。
- Azure OpenAI Service の接続とモデル情報
  - OpenAI API キー
  - OpenAI GPT-3.5 Turbo モデルのデプロイメント名
  - OpenAI API バージョン
- Python (この手順はバージョン 3.10.x でテストされています)

これらのデモには、Visual Studio Code と [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) を使用できます。


## パッケージのインストール

In [ ]:
!pip install semantic-kernel==0.4.5.dev0

## Azure OpenAI の設定
接続情報はセキュリティ面から直接記述するよりも、以下の情報を記載した `.env` ファイルからロードする方法がおすすめです。

### Azure OpenAI の場合
以下の情報が必要です。
```
AZURE_OPENAI_API_KEY="..."
AZURE_OPENAI_ENDPOINT="https://..."
AZURE_OPENAI_DEPLOYMENT_NAME="..."
```
### OpenAI の場合
以下の情報が必要です。
```
OPENAI_API_KEY="sk-..."
OPENAI_ORG_ID=""
```

In [ ]:
#AZURE_OPENAI_API_KEY = "Your OpenAI API Key"
#AZURE_OPENAI_ENDPOINT = "https://<Your OpenAI Service>.openai.azure.com/"
#AZURE_OPENAI_DEPLOYMENT_NAME = "gpt-35-turbo"
#AZURE_OPENAI_EMB_DEPLOYMENT_NAME = "text-embedding-ada-002"

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAITextEmbedding, AzureChatCompletion, AzureTextEmbedding

kernel = sk.Kernel()

# Azure OpenAI Service を使用するかどうか
useAzureOpenAI = True

# カーネルが使用する OpenAI サービスの設定
if useAzureOpenAI:
    deployment_name, api_key, endpoint = sk.azure_openai_settings_from_dot_env() # 環境変数からロードする場合
    azure_chat_service = AzureChatCompletion(deployment_name=deployment_name, endpoint=endpoint, api_key=api_key)
    #azure_text_embedding = AzureTextEmbedding(deployment_name=AZURE_OPENAI_EMB_DEPLOYMENT_NAME, endpoint=endpoint, api_key=api_key)
    kernel.add_chat_service("chat_completion", azure_chat_service)
    #kernel.add_text_embedding_generation_service("ada", azure_text_embedding)
else:
    # OpenAIの場合
    api_key, org_id = sk.openai_settings_from_dot_env() # 環境変数からロードする場合
    oai_text_service = OpenAIChatCompletion(ai_model_id="gpt-3.5-turbo", api_key=api_key, org_id=org_id)
    kernel.add_text_completion_service("chat_completion", oai_text_service)

## 要約
プロンプトを使って、コンテンツを要約するためのセマンティック関数を作ってみましょう。この関数は要約するテキストを入力として受け取ります。

In [ ]:
prompt = """
以下の説明を6歳の子供でも理解できるように分かりやすく要約してください。
子供が分からない単語は、分かりやすい言葉に置き換えてください。

{{$input}}
"""

summarize = kernel.create_semantic_function(prompt_template=prompt, max_tokens=400, temperature=0.0, top_p=0.5)


In [ ]:
input_text = """
中性子星は質量が太陽程度、直径20 km程度、大気の厚さはわずか1 m程度で、中性子が主な成分の天体である。
密度は太陽の10^14倍以上もあるとされている。およそ10^9 t/cm3とその桁外れに大きい密度のため、中性子星の
表面重力は地球の表面重力の2×10^11倍もの大きさがあり、脱出速度は 1/3 c にも達する。中性子星は大質量の
恒星の超新星爆発によってその中心核が圧縮された結果形成されるが、中性子星として存在できる質量にはトルマン
・オッペンハイマー・ヴォルコフ限界と呼ばれる上限値があり、それを超えるとブラックホールとなる。
上限の質量は、理論的に太陽質量の1.5倍から2.5倍の範囲にあると考えられており、2010年に約1.97倍の中性子星、
2013年には約2.01倍の中性子星が確認されている。下限は太陽質量の0.1倍から0.2倍程度。
"""

In [ ]:
summary = summarize(input_text)

print(summary)

## 履歴を考慮したチャットボット

In [ ]:
sk_prompt = """
日本の鎌倉時代の歴史に関する読解問題に答えるアシスタントです。
明確な指示を与えることも、答えがわからない場合は「わからない」と言うこともできます。

{{$history}}
User: {{$user_input}}
ChatBot: """

### セマンティック関数の登録

In [ ]:
chat_function = kernel.create_semantic_function(
    prompt_template=sk_prompt,
    function_name="ChatBot",
    max_tokens=2000,
    temperature=0.7,
    top_p=0.5)

### コンテキストを初期化

In [ ]:
context = kernel.create_new_context()
context["history"] = ""

### チャットを開始

In [ ]:
context["user_input"] = "源実朝ってどんな人"
bot_answer = await chat_function.invoke_async(context=context)
print(bot_answer)

### 出力で履歴を更新する

In [ ]:
context["history"] += f"\nUser: {context['user_input']}\nChatBot: {bot_answer}\n"
print(context["history"])

### 継続的なチャットのための関数を作成

In [ ]:
async def chat(input_text: str) -> None:
    # コンテキスト変数に新しいメッセージを保存
    print(f"User: {input_text}")
    context["user_input"] = input_text

    # ユーザーからのメッセージを処理し、回答を得る
    answer = await chat_function.invoke_async(context=context)

    # レスポンスを表示する
    print(f"ChatBot: {answer}")

    # Append the new interaction to the chat history
    context["history"] += f"\nUser: {input_text}\nChatBot: {answer}\n"

In [ ]:
await chat("源実朝が編纂した書物は何ですか？")

In [ ]:
await chat("源実朝が編纂したのは金槐和歌集ですよ")

In [ ]:
await chat("金槐和歌集はどんな書物ですか？")

In [ ]:
await chat("違いますね、源実朝のみの歌を集めた歌集です")

しばらくのチャットの後、`history` コンテキストに蓄積された完全なチャット履歴を確認します。

In [ ]:
print(context["history"])

## プラグイン（旧スキル）のロード
スキルとそのすべての関数をファイルからインポートします。`./samples/skills/FunSkill` ディレクトリの構造がそのままスキルの構成となります。

https://learn.microsoft.com/semantic-kernel/agents/plugins/?tabs=python

In [ ]:
# note: サンプルフォルダのスキルを使用する
skills_directory = "./samples/skills"

funFunctions = kernel.import_semantic_skill_from_directory(skills_directory, "FunSkill")

jokeFunction = funFunctions["Joke"]

ロードしたセマンティック関数は以下のようにして実行することができます。

In [ ]:
context = sk.ContextVariables()
context["style"] = "日本のお笑いの文化を理解したうえで"

result = jokeFunction("忍者について", variables=context)
print(result)

In [ ]:
# 非同期実行の場合
context = sk.ContextVariables()
context["style"] = "日本のお笑いの文化を理解したうえで"

result = await jokeFunction.invoke_async("忍者について", variables=context)
print(result)

## Basic プランナー
まずは基本的なプランナーを見てみましょう。`BasicPlanner` は JSON ベースの実行計画を生成し、提供された指示や質問を順次解決することを目的とし、順番に評価されます。

https://learn.microsoft.com/semantic-kernel/agents/planners/?tabs=python


### 計算機コアスキルのロード
プランナーは利用可能なスキルを知る必要があります。ここでは、ライブラリ内に事前定義されているコアスキルである `MathSkill` をロードします。

In [ ]:
from semantic_kernel.core_skills import MathSkill

kernel.import_skill(MathSkill(), "math")

In [ ]:
from semantic_kernel.planning.basic_planner import BasicPlanner

planner = BasicPlanner()

In [ ]:
ask = "202 と 990 の合計はいくらか？"

basic_plan = await planner.create_plan_async(ask, kernel)

print(basic_plan.generated_plan)

`BasicPlanner` が質問を受け取り、それを JSON ベースのプランに変換して、AI がカーネルが利用できるスキルを利用して、このタスクを解決する方法を説明していることがわかります。

上の実行計画でわかるように、AI はユーザーの要求を満たすためにどの関数を呼び出すべきかを決定しました。プランの各ステップの出力は、次の関数への入力となります。


### プランの実行
実行計画ができたので、それを実行してみましょう！ `BasicPlanner` には `execute_plan` という関数があります。

In [ ]:
results = await planner.execute_plan_async(basic_plan, kernel)
print(results)

## スキルを組み合わせる
### プランナーへのスキルの提供
プランナーは利用可能なスキルを知る必要があります。ここでは、ディスク上で定義した `SummarizeSkill` と `WriterSkill` にアクセスできるようにします。これには多くのセマンティック関数が含まれ、プランナーはそのサブセットをインテリジェントに選択します。

ネイティブ関数を含めることもできます。ここでは `TextSkill` を追加します。


In [ ]:
from semantic_kernel.core_skills.text_skill import TextSkill

skills_directory = "./samples/skills/"
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")
text_skill = kernel.import_skill(TextSkill(), "TextSkill")

In [ ]:
ask = """明日はバレンタインデーだ。デートのアイデアをいくつか考えないと。
彼女はシェークスピアが好きなので、彼のスタイルで書いてください。彼女はフランス語を話すので、フランス語で書きましょう。
あとテキストを大文字に変換しなさい。
"""

basic_plan = await planner.create_plan_async(ask, kernel)

print(basic_plan.generated_plan)

インラインスキルも定義して、プランナーが使用できるようにしましょう。必ず、関数名とスキル名を付けてください。

In [ ]:
sk_prompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
"""
shakespeareFunction = kernel.create_semantic_function(
    prompt_template=sk_prompt,
    function_name="shakespeare",
    skill_name="ShakespeareSkill",
    max_tokens=2000,
    temperature=0.8,
)

新たなプランを作成します。

In [ ]:
new_plan = await planner.create_plan_async(ask, kernel)
print(new_plan.generated_plan)

新しく作成したプランを実行します。

In [ ]:
results = await planner.execute_plan_async(new_plan, kernel)
print(results)